In [1]:
from langchain_community.llms import Ollama

# model 
llm = Ollama(model="gemma2:latest")

# chain 실행
response = llm.invoke("지구의 자전 주기는?")
response

'지구의 자전 주기는 약 **23시간 56분 4초**입니다. 이를 **하루**라고 합니다. \n\n다만, 실제로 지구가 한 바퀴 돌아서 완전히 같은 위치에 도달하는 데 걸리는 시간은 달라질 수 있습니다. 태양을 중심으로 공전하는 구름의 영향과 기타 요인 때문에 하루 길이가 약간 변동하기 때문입니다. \n'

# prompt template 이용해서 역할 부여

In [2]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# Ollama를 사용하여 Gemma2 모델 초기화
model = Ollama(model="gemma2:latest")

# ChatPromptTemplate 정의
prompt = ChatPromptTemplate.from_template(
    "당신은 금융 전문가 입니다. 질문에 친절하게 답을 해주세요. Let's start step by step. : {question}\n\n답변:"
)

# 체인 생성
chain = prompt | model | StrOutputParser()

# chain 호출
response = chain.invoke({"question": "금융 전문가가 하는 일과 직업 종류를 알려줘"})
print(response)

안녕하세요!  저는 금융 분야에 대해 많은 정보를 가지고 있습니다. 

금융 전문가는 여러분의 재정 목표를 달성하는 데 도움을 주기 위해 다양한 금융 상품과 서비스를 제공하는 사람입니다. 하지만 "금융 전문가"라는 제목은 너무 광범위해서  다양한 분야와 직업군을 포괄합니다. 

**몇 가지 예시를 들어 설명해 드릴게요:**

* **기획 및 투자**:
    * **투자 전문가 (Investment Advisor):** 개인 또는 기관의 자산을 관리하고 투자 계획을 수립하는 데 도움을 줍니다. 주식, 채권, 부동산 등 다양한 자산에 대한 전문 지식이 필요합니다.
    * **금융 분석가 (Financial Analyst):** 회사의 재무 상태를 분석하고 투자 가능성을 평가하며, 보고서를 작성하여 투자 결정에 도움을 줍니다.
* **대출 및 채권**:
    * **은행원 (Banker):** 개인과 기업에게 대출을 제공하고, 예금 계좌를 관리하는 등 은행의 금융 서비스를 제공합니다.
    * **보험 담당자 (Insurance Agent):** 보험 상품을 판매하고, 고객의 보험 필요성에 맞는 계획을 수립하는 데 도움을 줍니다.
* **정부 및 비영리**:
    * **금융 정책 전문가 (Financial Policy Analyst):** 정부 기관에서 금융 시장과 관련된 정책을 연구하고 제안하며,  경제 성장과 안정에 기여합니다.

이 외에도 다양한 분야의 금융 전문가들이 있습니다. 

어떤 분야에 대해 더 자세히 알고 싶으신가요? 🤔


# langchain, gemma2 이용해서 구글 뉴스 검색 후, 기능 제공
1. 뉴스 요약
2. 키워드 추출
3. 카테고리 분류
4. 뉴스에 대한 3가지 주요 point
5. 뉴스에 대한 인사이트까지 정리

In [5]:
def extract_long_texts(text, min_length=300):
    """
    주어진 텍스트에서 개행 문자로 분할한 후, 길이가 min_length 이상인 모든 텍스트 조각을 본문으로 추출
    :param text: 전체 뉴스 기사 텍스트
    :param min_length: 본문으로 간주할 최소 문자 수
    :return: 본문으로 간주된 텍스트 조각들
    """
    # 개행 문자(\n)를 기준으로 텍스트를 분할
    segments = text.split('\n')
    
    # 길이가 min_length 이상인 텍스트 조각을 찾음
    long_texts = [segment for segment in segments if len(segment) > min_length]
    
    # 모든 긴 텍스트 조각을 하나의 문자열로 결합
    return '\n'.join(long_texts)

In [ ]:
import pandas as pd
from langchain_community.document_loaders import WebBaseLoader

# CSV 파일에서 데이터프레임으로 읽어오기
news_df = pd.read_csv('news_data.csv')
news_df = news_df[:3]

# 'content'라는 새로운 컬럼 생성
news_df['content'] = ""

# 링크에서 원문 모두 스크래핑하여 'content' 컬럼에 추가
for i in range(len(news_df)):
    loader = WebBaseLoader(news_df['Original Link'][i])
    docs = loader.load()
    text = docs[0].page_content
    content_text = extract_long_texts(text)
    
    # content_text를 데이터프레임의 content 컬럼에 추가
    news_df.at[i, 'content'] = content_text

# 결과 확인
print(news_df.head())


In [11]:
news_df.content[2]

'                정부가 세금(현금) 대신 수령한 주식(물납주식)의 현금화를 위한 매각 활성 방안을 14일 발표했다. 상속세 대신 납부한 물납주식을 상속인이 되살 수 있도록 우선매수제도의 엄격한 신청 요건을 완화하고, 2회 이상 유찰된 물납주식은 상속인이 최대 50% 할인된 금액(단, 물납가 이상)에 매입할 수 있도록 하는 게 핵심이다. 최상목 부총리 겸 기획재정부장관은 14일 정부서울청사에서 26차 국유재산정책심의위원회를 주재하고 이러한 내용의 \'물납주식 매각 활성화 방안\'을 심의·의결했다. 물납은 현금 조달이 어렵다고 인정된 납세자를 위해 유가증권, 부동산 등의 다른 자산으로 세금을 납부할 수 있도록 한 제도다. 세 부담이 큰 상속세가 물납 대상이다.정부는 물납주식 매각을 위해 상속인 우선매수제도의 신청 문턱을 낮추기로 했다. 우선매수제는 가업 상속인이 물납주식을 재매입할 수 있도록 일정 기간 우선매수권을 부여하는 제도다. 2019년 도입된 우선매수제는 까다로운 요건 탓에 지금까지 신청 기업이 아예 없었다. 이에 정부는 \'중소기업\'과 \'매출액 3000억원 이하 중견기업\'으로 제한됐던 대상 기업을 모든 중소기업과 중견기업으로 확대하기로 했다.신청자 요건도 기존 \'대표이사 겸 최대주주\'에서 \'대표이사 또는 최대주주\'로 넓힌다. 피상속인(고인) 요건은 10년 이상 계속 경영을 하고 일정기간 대표이사로 재직하는 두 가지 요건을 동시에 충족해야 하는 것에서 한 가지만 부합하면 되는 것으로 완화한다.상속인의 물납주식 매입 부담도 대폭 줄인다. 기존에는 시장가치가 떨어진 유찰 물납주식을 상속인이 할인 혜택 없이 매입해야 했는데, 2회 이상 유찰된 물납주식의 경우 물납가보다는 높다는 전제하에 평가액 대비 20~50% 감액할 수 있는 규정을 마련하기로 했다. 2회 유찰될 경우 일반 입찰자는 20% 할인된 금액이 적용돼 상속인과의 형평성에 어긋난다는 문제를 해결하기 위해서다. 우선매수를 예약신청할 수 있는 기간도 물납허가일로부터 1년에서 3년으

In [16]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

# Ollama를 사용하여 Gemma2 모델 초기화
model = Ollama(model="gemma2:latest")

# ChatPromptTemplate 정의
prompt = PromptTemplate.from_template("""
    당신은 친절한 금융전문 뉴스 기자입니다. 
    아래의 포맷에 맞춰, "{text}"를 정리해주세요.
    
    1. 뉴스 제목
    2. 뉴스에서 중요한 3가지 포인트
    3. 뉴스에서 얻을 수 있는 중요 인사이트
""")

# 체인 생성
chain = prompt | model | StrOutputParser()

# chain 호출
response = chain.invoke({"text": news_df.content[2]})
print(response)


## 금융전문 뉴스 기자, 허세민 기자의 보도를 정리해 드립니다.

**1. 뉴스 제목:**  정부, 물납주식 현금화 활성 방안 발표...상속인 우선매수제 신청 완화 

**2. 중요 포인트 3가지:**
    * **물납주식 매각 활성화:** 정부는 상속세 대신 주식으로 납부한 물납주식 현금화를 위해  '물납주식 매각 활성화 방안'을 발표했습니다.
    * **상속인 우선매수제 완화:** 우선매수제 신청 요건을 완화하여 상속인들이 물납주식을 재매입하기 쉬운 환경을 조성합니다. (중소/중견기업 확대, 대표이사 또는 최대주주로 변경, 피상속인 요건 완화 등)
    * **유찰물납주식 할인 매입:**  2회 이상 유찰된 물납주식은 물납가보다 높은 평가액 대비 20~50% 감액하여 상속인이 매입할 수 있도록 합니다.

**3. 중요 인사이트:**
* **물납주식 문제의 심각성:** 물납주식 제때 매각하지 못해 시장가치가 훼손되면 '애물단지'로 전락하며 세수 증발로 이어집니다.  이는 정부 재정에 직접적인 영향을 미칠 수 있습니다.
* **정부의 적극적 대응:** 정부는 물납주식 현금화를 위한 다양한 노력을 기울이고 있으며, 이번 제도 개선을 통해 물납주식 매각이 활성화되고 세외수입 증대에 기여할 것으로 기대됩니다.
* **세무/시장 관점:**  개편된 제도는 상속인들에게 부담 경감과 투자 유치 효과를 가져올 수 있습니다. 또한, 물납주식 시장 활성화와 정부 세수 확보에도 긍정적인 영향을 줄 것으로 예상됩니다.







# 뉴스 원문 가져오는게 안되어서 아직 구동 X 코드

In [22]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from googlesearch import search
import requests
from bs4 import BeautifulSoup

# Gemma 2 모델을 사용하는 Ollama 인스턴스 생성
llm = Ollama(model="gemma2")

# 뉴스 검색 함수
def search_news(query, num_results=5):
    """
    구글 뉴스에서 주어진 쿼리로 뉴스를 검색합니다.
    :param query: 검색할 키워드
    :param num_results: 검색할 뉴스 개수
    :return: 검색된 뉴스 URL 리스트
    """
    search_results = search(query + " site:news.google.com", num_results=num_results)
    return list(search_results)

# 뉴스 내용 추출 함수
def extract_news_content(url):
    """
    주어진 URL에서 뉴스 내용을 추출합니다.
    :param url: 뉴스 URL
    :return: 추출된 뉴스 내용
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([p.text for p in paragraphs])
    return content[:1000]  # 첫 1000자만 반환

# 뉴스 요약 프롬프트 템플릿
summarize_template = """
다음 뉴스를 200자 이내로 요약해주세요:

{news_content}

요약:
"""
summarize_prompt = PromptTemplate(template=summarize_template, input_variables=["news_content"])
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

# 키워드 추출 및 카테고리 분류 프롬프트 템플릿
categorize_template = """
다음 뉴스 요약에서 주요 키워드 5개를 추출하고, 뉴스의 카테고리를 분류해주세요:

{news_summary}

형식:
키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5
카테고리: [경제, 주식, 부동산, 국제금융, 기업, 정책 중 하나 선택]
"""
categorize_prompt = PromptTemplate(template=categorize_template, input_variables=["news_summary"])
categorize_chain = LLMChain(llm=llm, prompt=categorize_prompt)

# 인사이트 도출 프롬프트 템플릿
insight_template = """
다음 뉴스 요약을 바탕으로 3가지 주요 포인트와 전반적인 인사이트를 제공해주세요:

{news_summary}

형식:
주요 포인트:
1. 
2. 
3. 

인사이트:
"""
insight_prompt = PromptTemplate(template=insight_template, input_variables=["news_summary"])
insight_chain = LLMChain(llm=llm, prompt=insight_prompt)

# 메인 함수
def analyze_financial_news(query):
    """
    금융 뉴스를 검색하고 분석하는 메인 함수입니다.
    :param query: 검색할 키워드
    """
    print(f"'{query}'에 대한 뉴스를 검색 중입니다...")
    news_urls = search_news(query)
    
    for url in news_urls:
        print(f"\n뉴스 URL: {url}")
        
        # 뉴스 내용 추출
        content = extract_news_content(url)
        
        # 뉴스 요약
        summary = summarize_chain.run(content)
        print(f"\n요약:\n{summary}")
        
        # 키워드 추출 및 카테고리 분류
        categorization = categorize_chain.run(summary)
        print(f"\n키워드 및 카테고리:\n{categorization}")
        
        # 인사이트 도출
        insights = insight_chain.run(summary)
        print(f"\n인사이트:\n{insights}")
        
        print("\n" + "="*50 + "\n")

# 사용 예시
if __name__ == "__main__":
    analyze_financial_news("한국 금융시장 동향")

'한국 금융시장 동향'에 대한 뉴스를 검색 중입니다...

뉴스 URL: https://news.google.com/__i/rss/rd/articles/CBMiM2h0dHBzOi8va3IuaW52ZXN0aW5nLmNvbS9lcXVpdGllcy90b3Atc3RvY2stZ2FpbmVyc9IBAA?oc=5

요약:
##  뉴스 요약을 위해 뉴스 내용을 입력해 주세요! 😊 
 
 뉴스 내용을 알려주시면, 200자 이내로 정리해서 드릴게요! 📰✨  


키워드 및 카테고리:
알겠습니다! 뉴스 내용을 입력해주세요. 😊  

내용을 알려주시면 200자 이내로 요약하고, 주요 키워드 5개와 카테고리를 분류해 드리겠습니다.  📰✨ 




인사이트:
뉴스 요약을 입력해 주세요! 😊  제가 멋진 요약과 인사이트를 드리도록 기대됩니다! ✨📰  


예시: "국내 최초로 AI 작곡 프로그램 개발..." 와 같이 뉴스 내용을 알려주세요. 😄  



뉴스 URL: https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3LmFzaWF0aW1lLmNvLmtyL2FydGljbGUvMjAyMzAzMDk1MDAwMjTSAQA?oc=5

요약:
제공된 뉴스가 없습니다. 뉴스를 입력해 주시면 200자 이내로 요약해 드리겠습니다! 😊  


키워드 및 카테고리:
알겠습니다! 뉴스를 입력해주시면  200자 이내로 요약해 드리고, 키워드 5개와 카테고리를 분류해 드리겠습니다. 😊  


기다리고 있어요! 👍  


인사이트:
뉴스를 입력해주세요!  📰✨ 

제가 뉴스를 읽고 핵심 포인트와 흥미로운 통찰력을 분석해서 드릴게요. 😊   


**예시:**

"삼성전자, 신형 스마트폰 출시 예정...AI 기능 강화!"

그런 식으로 뉴스 주제를 알려주세요! 👍  




뉴스 URL: https://news.google.com/__i/rss/rd/articles/CBMiNmh0dHBzOi8vd3d3Lmhhbmt5dW5nLmNvbS9lY29ub215L2FydGl

# 다른 방법으로 시도 

In [24]:
# 라이브러리 설치
# !pip install -q langchain-openai google-search-results